In [78]:
import os, sys

# if os.path.exists('analysis'):
#     os.system('rm -rf analysis')

# !git clone https://github.com/element-fi/analysis.git

parent_dir = os.path.join(os.getcwd(), os.pardir)
sys.path.insert(1, os.path.join(parent_dir, "src"))

import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

pd.set_option("float_format",'{:,.0f}'.format)

In [79]:
from elfpy.pricing_models import YieldSpacev2PricingModel

In [80]:
target_liquidity_usd = 10*1000 # $10k
market_price = 1
apy = 5
days_remaining = 365/2
time_stretch = 22.58
mdl = YieldSpacev2PricingModel(verbose=True)
(base_asset_reserves, token_asset_reserves, total_liquidity, spot_price) = mdl.calc_liquidity(target_liquidity_usd=target_liquidity_usd, \
    market_price=market_price, apy=apy, days_remaining=days_remaining, time_stretch=time_stretch)
[base_asset_reserves_usd, token_asset_reserves_usd, total_liquidity_usd] = [x * market_price for x in [base_asset_reserves, token_asset_reserves, total_liquidity]]
total_supply = base_asset_reserves + token_asset_reserves
time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
calc_apy = mdl.calc_apy_from_reserves(base_asset_reserves, token_asset_reserves, total_supply, time_remaining, time_stretch)
calc_hpr = calc_apy * days_remaining / 365
print(f"input: target_liquidity_usd={target_liquidity_usd}, market_price={market_price}, apy={apy}, days_remaining={days_remaining}, time_stretch={time_stretch}")
# print(f"spot price = {spot_price}")
# print('output in ETH terms:')
# print(f"base_asset_reserves: {base_asset_reserves}\ntoken_asset_reserves: {token_asset_reserves}\ntotal_liquidity: {total_liquidity}")
# print('output in USD terms:')
print(f"\
base_asset_reserves:  ${base_asset_reserves_usd:6,.0f}\n\
token_asset_reserves: ${token_asset_reserves_usd*spot_price:6,.0f} ({token_asset_reserves_usd:,.0f} PTs at {spot_price:.4f})\n\
total_liquidity:      ${total_liquidity_usd:6,.0f} ({base_asset_reserves_usd+token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
APR:                   {calc_apy:6,.4f}%"
)
initial_price = spot_price
initial_days_remaining = days_remaining
initial_apy = calc_apy
initial_hpr = calc_hpr
initial_base_asset_reserves = base_asset_reserves
initial_token_asset_reserves = token_asset_reserves

calc_base_asset_reserves result: 5000.17263424107
base_asset_reserves=5000.086315630432, token_asset_reserves=5124.911526478807, total_supply=10,000.0000(10,000 USD), apy=4.999999999999981
input: target_liquidity_usd=10000, market_price=1, apy=5, days_remaining=182.5, time_stretch=22.58
base_asset_reserves:  $ 5,000
token_asset_reserves: $ 5,000 (5,125 PTs at 0.9756)
total_liquidity:      $10,000 (10,125.00 tokens, base + PT)
APR:                   5.0000%


In [81]:
trade_in_usd = 1000
token_out = 'fyt'
token_in = 'fyt' if token_out == 'base' else 'base'
in_ = trade_in_usd / market_price
total_supply = base_asset_reserves + token_asset_reserves
in_reserves = token_asset_reserves + total_supply if token_out == 'base' else base_asset_reserves
out_reserves = base_asset_reserves if token_out == 'base' else token_asset_reserves + total_supply
fee_percent = 0.1
time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
(without_fee_or_slippage, with_fee, without_fee, fee) = mdl.calc_out_given_in(in_=in_,in_reserves=in_reserves,out_reserves=out_reserves,token_out=token_out,fee_percent=fee_percent,time_remaining=time_remaining,share_price=1,init_share_price=1)
[without_fee_or_slippage_usd, with_fee_usd, without_fee_usd, fee_usd] = [x * market_price for x in [without_fee_or_slippage, with_fee, without_fee, fee]]
new_base_asset_reserves = base_asset_reserves - with_fee if token_out == 'base' else base_asset_reserves + in_
new_token_asset_reserves = token_asset_reserves + in_ if token_out == 'base' else token_asset_reserves - with_fee
new_total_supply = new_base_asset_reserves + new_token_asset_reserves
new_spot_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, new_total_supply, time_remaining)
new_total_liquidity = new_base_asset_reserves + new_token_asset_reserves * new_spot_price
[new_base_asset_reserves_usd, new_token_asset_reserves_usd, new_total_liquidity_usd] = [x * market_price for x in [new_base_asset_reserves, new_token_asset_reserves, new_total_liquidity]]
new_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, new_total_supply, time_remaining, time_stretch)
print(f"trade:\n\
  in:                       {in_*market_price:8,.2f} {token_in}\n\
  without_fee_or_slippage:  {without_fee_or_slippage_usd:8,.2f} {token_out}\n\
  without_fee:              {without_fee_usd:8,.2f} {token_out}\n\
  slippage:                 {without_fee_or_slippage_usd-without_fee_usd:8,.2f} {token_out}\n\
  fee:                      {fee_usd:8,.2f} {token_out} (fee applied to {fee_usd/fee_percent:,.2f})\n\
  with_fee:                 {with_fee_usd:8,.2f} {token_out}")
print(f"new reserves:\n\
  base_asset_reserves:  ${new_base_asset_reserves_usd:6,.0f}\n\
  token_asset_reserves: ${new_token_asset_reserves_usd*new_spot_price:6,.0f} ({new_token_asset_reserves_usd:,.0f} PTs at {new_spot_price:.4f})\n\
  total_liquidity:      ${new_total_liquidity_usd:6,.0f} ({new_base_asset_reserves_usd+new_token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
  APR:                   {new_apy:6,.4f}%")


trade:
  in:                       1,000.00 base
  without_fee_or_slippage:  1,025.00 fyt
  without_fee:              1,022.09 fyt
  slippage:                     2.91 fyt
  fee:                          2.21 fyt (fee applied to 22.09)
  with_fee:                 1,019.89 fyt
new reserves:
  base_asset_reserves:  $ 6,000
  token_asset_reserves: $ 4,027 (4,105 PTs at 0.9811)
  total_liquidity:      $10,027 (10,105.11 tokens, base + PT)
  APR:                   3.8550%


In [82]:

spot_price = initial_price
day_bump = -1/24
days_remaining = initial_days_remaining
base_asset_reserves = initial_base_asset_reserves + trade_in_usd / market_price
# token_asset_reserves = token_asset_reserves - with_fee # using out amount from yieldspace trade above
token_asset_reserves = initial_token_asset_reserves - trade_in_usd / market_price / spot_price # same result using no slippage or fee
total_supply = new_base_asset_reserves + new_token_asset_reserves
time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
spot_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining)
calc_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining, time_stretch)
calc_hpr = calc_apy * days_remaining / 365
starting_apy = calc_apy
starting_hpr = calc_hpr
starting_price = spot_price
print(f"calc_apy: {calc_apy:6,.4f}%")
# while (spot_price > initial_price):
while (calc_hpr < initial_hpr):
    day_bump += 1/24
    days_remaining = initial_days_remaining + day_bump
    time_remaining = mdl.days_to_time_remaining(days_remaining, time_stretch)
    spot_price = mdl.calc_spot_price_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining)
    calc_apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining, time_stretch)
    calc_hpr = calc_apy * days_remaining / 365
    print(f"day_bump: {day_bump}, price: {spot_price:6,.4f}, apy: {calc_apy:6,.4f}, reserves: x={new_base_asset_reserves:6,.0f} y={new_token_asset_reserves:6,.0f}")
    if day_bump>1e6 or day_bump<-1000:
        break
total_liquidity = new_base_asset_reserves + new_token_asset_reserves * spot_price
apy = mdl.calc_apy_from_reserves(new_base_asset_reserves, new_token_asset_reserves, total_supply, time_remaining, time_stretch)
[new_base_asset_reserves_usd, new_token_asset_reserves_usd, total_liquidity_usd] = [x * market_price for x in [new_base_asset_reserves, new_token_asset_reserves, total_liquidity]]

calc_apy: 3.8550%
day_bump: 0.0, price: 0.9811, apy: 3.8550, reserves: x= 6,000 y= 4,105
day_bump: 0.041666666666666664, price: 0.9811, apy: 3.8550, reserves: x= 6,000 y= 4,105
day_bump: 0.08333333333333333, price: 0.9811, apy: 3.8550, reserves: x= 6,000 y= 4,105
day_bump: 0.125, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.16666666666666666, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.20833333333333331, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.24999999999999997, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.29166666666666663, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.3333333333333333, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.375, price: 0.9811, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.4166666666666667, price: 0.9810, apy: 3.8551, reserves: x= 6,000 y= 4,105
day_bump: 0.45833333333333337, price: 0.9810, apy: 3.8551, reserves: x

In [83]:

print(f"day_bump: {day_bump}, days_remaining={days_remaining:,.0f}(years={days_remaining/365:.2f}), time_stretch={time_stretch}")
# print(f"spot price = {spot_price}")
# print('output in ETH terms:')
# print(f"base_asset_reserves: {base_asset_reserves}\ntoken_asset_reserves: {token_asset_reserves}\ntotal_liquidity: {total_liquidity}")
# print('output in USD terms:')
print(f"\
base_asset_reserves:  ${new_base_asset_reserves:6,.0f}\n\
token_asset_reserves: ${new_token_asset_reserves_usd*spot_price:6,.0f} ({new_token_asset_reserves_usd:,.0f} PTs at {spot_price:.4f})\n\
total_liquidity:      ${total_liquidity_usd:6,.0f} ({new_base_asset_reserves_usd+new_token_asset_reserves_usd:,.2f} tokens, base + PT)\n\
APR:                   {apy:6,.4f}% (initial={initial_apy:6,.4f}% post-swap={starting_apy:6,.4f}%)\n\
HPR:                   {calc_hpr:6,.4f}% (initial={initial_hpr:6,.4f}% post-swap={starting_hpr:6,.4f}%)\n\
price:                 {spot_price:6,.5f} (initial={initial_price:6,.5f} post-swap={starting_price:6,.5f})"
)
print(f"-log(original spot price)/log((new y reserves)/(new x reserves)): {-np.log(initial_price)/np.log((new_token_asset_reserves+total_liquidity_usd)/new_base_asset_reserves)}")
print(f"                                                  *365*t_stretch: {-(np.log(initial_price)/np.log((new_token_asset_reserves+total_liquidity_usd)/new_base_asset_reserves))*(365)*time_stretch}")

day_bump: 53.541666666665805, days_remaining=236(years=0.65), time_stretch=22.58
base_asset_reserves:  $ 6,000
token_asset_reserves: $ 4,005 (4,105 PTs at 0.9756)
total_liquidity:      $10,005 (10,105.11 tokens, base + PT)
APR:                   3.8659% (initial=5.0000% post-swap=3.8550%)
HPR:                   2.5000% (initial=2.5000% post-swap=1.9275%)
price:                 0.97561 (initial=0.97561 post-swap=0.98109)
-log(original spot price)/log((new y reserves)/(new x reserves)): 0.02887649923832528
                                                  *365*t_stretch: 237.99144377250542
